In [1]:
from pathlib import Path
import os
from collections import Counter
import numpy as np
import time
from tqdm.notebook import tqdm

np.random.seed(8080)

data_path = Path("/content/drive/My Drive/Adv Projects in ML/data")
print(data_path)
print(os.listdir(data_path))

os.chdir("/content/drive/My Drive/Adv Projects in ML/")

!nvidia-smi

/content/drive/My Drive/Adv Projects in ML/data
['train.lang2', 'unaligned.en', 'unaligned.fr', 'train.lang1', 'split_train.lang1', 'split_val.lang2', 'split_train.lang2', 'split_val.lang1', 'unalignedtry.en', 'split_train_unaligned_tokenized_rempunc.en', 'split_val_unaligned_tokenized_rempunc.en', 'split_train_unaligned_tokenized.en', 'split_val_unaligned_tokenized.en', 'split_train_unaligned_tokenized_rempunc.fr', 'split_val_unaligned_tokenized_rempunc.fr', 'split_train_unaligned_tokenized.fr', 'split_val_unaligned_tokenized.fr', 'bpe', 'unaligned_tokenized_rempunc.en']
Wed Apr 15 06:05:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Co

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
from transformer import Transformer, CustomSchedule, create_masks

print("Tensorflow version " + tf.__version__)

tf.random.set_seed(8080)
# make sure numpy seeded

Tensorflow version 2.2.0-rc2


In [3]:
# read data
with open(data_path/"split_train.lang1","r") as f:
    english = f.read()
print(len(english.split("\n")), english[:200])
    
with open(data_path/"split_train.lang2","r") as f:
    french = f.read()
print(len(french.split("\n")), french[:200])

with open(data_path/"split_val.lang1","r") as f:
    english_val = f.read()
print(len(english_val.split("\n")), english_val[:200])

with open(data_path/"split_val.lang2","r") as f:
    french_val = f.read()
print(len(french_val.split("\n")), french_val[:200])

with open("predictions_english_st.txt","r") as f:
    french_st = f.read().strip()
print(len(french_st.split("\n")), french_st[:200])

with open(data_path/"unaligned.en","r") as f:
    english_monolingual = f.read().strip()
# english_monolingual = english_monolingual.split("\n")[:len(french_st.split("\n"))]
print(len(english_monolingual.split("\n")), english_monolingual[:200])

# create vocab
english_vocab = list(set(english.replace("\n", " <eos> ").split()))
french_vocab = list(set(french.replace("\n", " <eos> ").split()))
len(english_vocab), len(french_vocab)

english_counter = Counter(english.replace("\n", " <eos> ").split())
french_counter = Counter(french.replace("\n", " <eos> ").split())
len(english_counter), len(french_counter)

english_counter.update({"<unk>":0})
french_counter.update({"<unk>":0})
english_counter.update({"<start>":0})
french_counter.update({"<start>":0})
len(english_counter), len(french_counter)

english_vocab = list(english_counter.keys())
french_vocab = list(french_counter.keys())

# # trim vocab to 10k+2, 12k+2
# english_vocab = ["<start>","<unk>"]
# for i in english_counter.most_common(10000):
#   english_vocab.append(i[0])
# french_vocab = ["<start>","<unk>"]
# for i in french_counter.most_common(12000):
#   french_vocab.append(i[0])

english_word2id = {}
english_id2word = {}
french_word2id = {}
french_id2word = {}

# start enumerate from 1 so that 0 is reserved for padding seqs 
for i, w in enumerate(english_vocab, start=1):
  english_word2id[w] = i
  english_id2word[i] = w

for i, w in enumerate(french_vocab, start=1):
  french_word2id[w] = i
  french_id2word[i] = w

len(english_word2id), len(english_id2word), len(french_word2id), len(french_id2word)

def transform_data(english_lang1, french_lang2):
  english_lines = english_lang1.split("\n")
  french_lines = french_lang2.split("\n")

  data_english = []
  data_french = []

  for line in english_lines:
    line2id = [english_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(english_word2id[word])
      except:
        line2id.append(english_word2id["<unk>"])
    line2id.append(english_word2id["<eos>"])
    data_english.append(line2id)

  for line in french_lines:
    line2id = [french_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(french_word2id[word])
      except:
        line2id.append(french_word2id["<unk>"])
    line2id.append(french_word2id["<eos>"])
    data_french.append(line2id)

  print(len(data_english), len(data_french))
  return data_english, data_french

data_english, data_french = transform_data(english, french)
data_english_val, data_french_val = transform_data(english_val, french_val)

data_english_monolingual, data_french_st = transform_data(english_monolingual, french_st)
data_english_monolingual = data_english_monolingual[:len(data_french_st)]

english_id2word[54], len(data_english), len(data_french), len(data_english_val), len(data_french_val), len(data_english_monolingual), len(data_french_st)

8800 as mr de castro is not present mr le foll who is replacing mr de castro has the floor
on the other hand if you 're visiting an underdeveloped country and 25 dollars buys you a gourmet meal it 's exorb
8800 Comme M. De Castro est absent , M. Le Foll , qui le remplace , a la parole .
D' un autre côté , si vous êtes dans un pays en voie de développement , où 25 dollars peuvent vous obtenir un repas de luxe
2200 what action does the council intend to take in the face of this seriously discriminatory attitude which runs contrary to the principles of the eu
where would you like to go next
if that were not enoug
2200 Quelles mesures le Conseil compte-t-il adopter face à cette attitude qui constitue une grave discrimination et est contraire aux principes sur lesquels l' Union européenne est fondée ?
Où souhaiteriez
200000 Pour la deuxième fois , nous avons des frais d’ être prises par rapport à des petites entreprises et que c' est vrai .
La dernière séance , j' ai été dit que j' étais le

('gas', 8800, 8800, 2200, 2200, 200000, 200000)

In [0]:
np.savez("data_and_vocab_st.npz", data_english=data_english, data_french=data_french, data_english_val=data_english_val, data_french_val=data_french_val, 
         data_english_monolingual=data_english_monolingual, data_french_st=data_french_st,
         english_word2id=english_word2id, english_id2word=english_id2word, french_word2id=french_word2id,french_id2word=french_id2word)

In [5]:
BUFFER_SIZE = len(data_english)
BATCH_SIZE = 64
EPOCHS = 5
print("No. of batches: ", np.ceil(len(data_english_monolingual)/BATCH_SIZE))
print("No. of batches: ", np.ceil(len(data_english)/BATCH_SIZE))
repeat_factor = len(data_english_monolingual) // len(data_english) + 1
print(repeat_factor)

# transformer hyperparams
num_layers = 1
d_model = 256
dff = 256
num_heads = 8
input_vocab_size = len(english_vocab) + 1
target_vocab_size = len(french_vocab) + 1
dropout_rate = 0.4
p_wd_st = 0.3
pe_input = max(max([len(i) for i in data_english]),max([len(i) for i in data_english_val]),max([len(i) for i in data_english_monolingual]))
pe_target = max(max([len(i) for i in data_french]),max([len(i) for i in data_french_val]),max([len(i) for i in data_french_st]))

# pe_input = 200
# pe_target = 230
pe_input, pe_target

No. of batches:  3125.0
No. of batches:  138.0
23


(112, 222)

In [0]:
tensor_train = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(data_english, padding='post'),
    tf.keras.preprocessing.sequence.pad_sequences(data_french, padding='post')
)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=False)
tensor_val = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(data_english_val, padding='post'),
    tf.keras.preprocessing.sequence.pad_sequences(data_french_val, padding='post')
)).batch(BATCH_SIZE, drop_remainder=False)

In [0]:
class DatasetGenerator_ST(tf.data.Dataset):
  def _generator():
    inp_pad = tf.keras.preprocessing.sequence.pad_sequences(data_english_monolingual, padding='post').shape[1]
    tar_pad = tf.keras.preprocessing.sequence.pad_sequences(data_french_st, padding='post').shape[1]
    indexes = np.arange(len(data_french_st))
    np.random.shuffle(indexes)
    data1 = np.array(data_english_monolingual)[indexes]
    data2 = np.array(data_french_st)[indexes]
    for i in range(len(data2)):
      tar = data2[i]
      tar = np.pad(tar, (0,tar_pad-len(tar)))
      aug = data1[i]
      if np.random.choice(['drop','swap']) == 'drop':
        drop_idxs = np.random.binomial(1,0.3,len(aug))
        drop_idxs = np.where(drop_idxs==1)
        aug = np.delete(aug,drop_idxs)
      else:
        swap_idx = np.random.choice(np.arange(1,len(aug)))
        tmp = aug[swap_idx]
        aug[swap_idx] = aug[swap_idx-1]
        aug[swap_idx-1] = tmp
      aug = np.pad(aug, (0,inp_pad-len(aug)))
      yield aug, tar

  def __new__(cls):
      return tf.data.Dataset.from_generator(
          cls._generator,
          output_types=(tf.dtypes.int32,tf.dtypes.int32),
          output_shapes=(None,None)
      )
tensor_st = DatasetGenerator_ST().batch(BATCH_SIZE, drop_remainder=False).prefetch(tf.data.experimental.AUTOTUNE)


In [8]:
transformer = Transformer(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff, 
    input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size, 
    pe_input=pe_input, pe_target=pe_target, rate=dropout_rate)

temp_input = tf.random.uniform((BATCH_SIZE, pe_input), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((BATCH_SIZE, pe_target), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

TensorShape([64, 222, 16315])

In [9]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Encoder)            multiple                  3560704   
_________________________________________________________________
decoder (Decoder)            multiple                  4836096   
_________________________________________________________________
dense_16 (Dense)             multiple                  4192955   
Total params: 12,589,755
Trainable params: 12,589,755
Non-trainable params: 0
_________________________________________________________________


In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_loss = tf.keras.metrics.Mean(name='loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='val_accuracy')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [0]:
experiment_number = "st_3_"

checkpoint_path = "./checkpoints/train"+experiment_number

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

writer_train = tf.summary.create_file_writer("log_dir/"+experiment_number+"_train")
writer_val = tf.summary.create_file_writer("log_dir/"+experiment_number+"_val")

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [0]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

@tf.function(input_signature=train_step_signature)
def val_step(inp, tar):
  
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  predictions, _ = transformer(inp, tar_inp, 
                                False, 
                                enc_padding_mask, 
                                combined_mask, 
                                dec_padding_mask)
  loss = loss_function(tar_real, predictions)
  
  val_loss(loss)
  val_accuracy(tar_real, predictions)


In [13]:
best_val_loss = np.inf

for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  print("training ST data")
  tensor_st = DatasetGenerator_ST().batch(BATCH_SIZE, drop_remainder=False).prefetch(tf.data.experimental.AUTOTUNE)
  for (batch, (inp, tar)) in tqdm(enumerate(tensor_st), total=len(data_english_monolingual)//BATCH_SIZE+1):
    train_step(inp, tar)
    if batch % 100 == 0:
      print ('Epoch {} Batch {} Training Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))

  for iteration_i in range(repeat_factor):

    train_loss.reset_states()
    train_accuracy.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()

    print("training Parallel data")

    for (batch, (inp, tar)) in tqdm(enumerate(tensor_train)):
      train_step(inp, tar)
      if batch % 50 == 0:
        print ('Epoch {} iteration_i {} Batch {} Training Loss {:.4f} Accuracy {:.4f}'.format(
            epoch + 1, iteration_i, batch, train_loss.result(), train_accuracy.result()))
      
    print ('Epoch {} iteration_i {} Training Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                  iteration_i,
                                                  train_loss.result(), 
                                                  train_accuracy.result()))
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

    with writer_train.as_default():
      tf.summary.scalar('train_loss', train_loss.result(), step=epoch)

    print("validating")
    for (batch, (inp, tar)) in enumerate(tensor_val):
      val_step(inp, tar)
    
    print ('Epoch {} iteration_i {} Validation Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                              iteration_i,
                                              val_loss.result(), 
                                              val_accuracy.result()))
    if best_val_loss > val_loss.result():
      best_val_loss = val_loss.result()
      ckpt_save_path = ckpt_manager.save()
      print ('Saving checkpoint for epoch {} iteration_i {} at {}'.format(epoch+1,
                                                          iteration_i,
                                                          ckpt_save_path))
    with writer_val.as_default():
      tf.summary.scalar('val_loss', val_loss.result(), step=epoch)

training ST data


Epoch 1 Batch 0 Training Loss 9.7294 Accuracy 0.0000
Epoch 1 Batch 100 Training Loss 9.5965 Accuracy 0.0005
Epoch 1 Batch 200 Training Loss 9.1716 Accuracy 0.0022
Epoch 1 Batch 300 Training Loss 8.5456 Accuracy 0.0030
Epoch 1 Batch 400 Training Loss 7.9554 Accuracy 0.0035
Epoch 1 Batch 500 Training Loss 7.5317 Accuracy 0.0047
Epoch 1 Batch 600 Training Loss 7.1994 Accuracy 0.0062
Epoch 1 Batch 700 Training Loss 6.8948 Accuracy 0.0079
Epoch 1 Batch 800 Training Loss 6.6048 Accuracy 0.0097
Epoch 1 Batch 900 Training Loss 6.3348 Accuracy 0.0115
Epoch 1 Batch 1000 Training Loss 6.0867 Accuracy 0.0132
Epoch 1 Batch 1100 Training Loss 5.8643 Accuracy 0.0148
Epoch 1 Batch 1200 Training Loss 5.6647 Accuracy 0.0162
Epoch 1 Batch 1300 Training Loss 5.4850 Accuracy 0.0176
Epoch 1 Batch 1400 Training Loss 5.3239 Accuracy 0.0188
Epoch 1 Batch 1500 Training Loss 5.1778 Accuracy 0.0199
Epoch 1 Batch 1600 Training Loss 5.0457 Accuracy 0.0210
Epoch 1 Batch 1700 Training Loss 4.9247 Accuracy 0.0220
Epoc

Epoch 1 iteration_i 0 Batch 0 Training Loss 6.2153 Accuracy 0.0558
Epoch 1 iteration_i 0 Batch 50 Training Loss 4.9058 Accuracy 0.0594
Epoch 1 iteration_i 0 Batch 100 Training Loss 4.7728 Accuracy 0.0611

Epoch 1 iteration_i 0 Training Loss 4.7054 Accuracy 0.0615
Time taken for 1 epoch: 765.4688374996185 secs

validating
Epoch 1 iteration_i 0 Validation Loss 4.5016 Accuracy 0.0685
Saving checkpoint for epoch 1 iteration_i 0 at ./checkpoints/trainst_3_/ckpt-1
training Parallel data


Epoch 1 iteration_i 1 Batch 0 Training Loss 4.1501 Accuracy 0.0769
Epoch 1 iteration_i 1 Batch 50 Training Loss 4.1392 Accuracy 0.0680
Epoch 1 iteration_i 1 Batch 100 Training Loss 4.1260 Accuracy 0.0682

Epoch 1 iteration_i 1 Training Loss 4.1224 Accuracy 0.0687
Time taken for 1 epoch: 791.8378891944885 secs

validating
Epoch 1 iteration_i 1 Validation Loss 4.4314 Accuracy 0.0715
Saving checkpoint for epoch 1 iteration_i 1 at ./checkpoints/trainst_3_/ckpt-2
training Parallel data


Epoch 1 iteration_i 2 Batch 0 Training Loss 3.5178 Accuracy 0.0706
Epoch 1 iteration_i 2 Batch 50 Training Loss 3.7842 Accuracy 0.0736
Epoch 1 iteration_i 2 Batch 100 Training Loss 3.7886 Accuracy 0.0736

Epoch 1 iteration_i 2 Training Loss 3.8003 Accuracy 0.0739
Time taken for 1 epoch: 817.3789081573486 secs

validating
Epoch 1 iteration_i 2 Validation Loss 4.4193 Accuracy 0.0739
Saving checkpoint for epoch 1 iteration_i 2 at ./checkpoints/trainst_3_/ckpt-3
training Parallel data


...

Epoch 5 iteration_i 18 Batch 0 Training Loss 0.4633 Accuracy 0.1719
Epoch 5 iteration_i 18 Batch 50 Training Loss 0.4859 Accuracy 0.1846
Epoch 5 iteration_i 18 Batch 100 Training Loss 0.5110 Accuracy 0.1838

Epoch 5 iteration_i 18 Training Loss 0.5247 Accuracy 0.1823
Time taken for 1 epoch: 1201.9836168289185 secs

validating
Epoch 5 iteration_i 18 Validation Loss 6.6094 Accuracy 0.0817
training Parallel data


Epoch 5 iteration_i 19 Batch 0 Training Loss 0.4316 Accuracy 0.1946
Epoch 5 iteration_i 19 Batch 50 Training Loss 0.4856 Accuracy 0.1860
Epoch 5 iteration_i 19 Batch 100 Training Loss 0.4964 Accuracy 0.1839

Epoch 5 iteration_i 19 Training Loss 0.5090 Accuracy 0.1830
Time taken for 1 epoch: 1226.8338775634766 secs

validating
Epoch 5 iteration_i 19 Validation Loss 6.6705 Accuracy 0.0819
training Parallel data


Epoch 5 iteration_i 20 Batch 0 Training Loss 0.5278 Accuracy 0.1848
Epoch 5 iteration_i 20 Batch 50 Training Loss 0.4911 Accuracy 0.1842
Epoch 5 iteration_i 20 Batch 100 Training Loss 0.4966 Accuracy 0.1841

Epoch 5 iteration_i 20 Training Loss 0.5054 Accuracy 0.1828
Time taken for 1 epoch: 1251.6097049713135 secs

validating
Epoch 5 iteration_i 20 Validation Loss 6.6985 Accuracy 0.0814
training Parallel data


Epoch 5 iteration_i 21 Batch 0 Training Loss 0.4912 Accuracy 0.1860
Epoch 5 iteration_i 21 Batch 50 Training Loss 0.4714 Accuracy 0.1834
Epoch 5 iteration_i 21 Batch 100 Training Loss 0.4844 Accuracy 0.1844

Epoch 5 iteration_i 21 Training Loss 0.4952 Accuracy 0.1835
Time taken for 1 epoch: 1276.3148612976074 secs

validating
Epoch 5 iteration_i 21 Validation Loss 6.7422 Accuracy 0.0819
training Parallel data


Epoch 5 iteration_i 22 Batch 0 Training Loss 0.4222 Accuracy 0.1971
Epoch 5 iteration_i 22 Batch 50 Training Loss 0.4570 Accuracy 0.1836
Epoch 5 iteration_i 22 Batch 100 Training Loss 0.4751 Accuracy 0.1848

Epoch 5 iteration_i 22 Training Loss 0.4833 Accuracy 0.1840
Time taken for 1 epoch: 1301.1362941265106 secs

validating
Epoch 5 iteration_i 22 Validation Loss 6.7671 Accuracy 0.0814


Evaluate best model

In [0]:
# load model

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [0]:
transformer

In [14]:
val_loss.reset_states()
val_accuracy.reset_states()
  
for (batch, (inp, tar)) in tqdm(enumerate(tensor_val)):
  val_step(inp, tar)
  
print ('Validation Loss {:.4f} Accuracy {:.4f}'.format(
                                          val_loss.result(), 
                                          val_accuracy.result()))
  


Validation Loss 6.7671 Accuracy 0.0814


In [0]:
def generate_predictions(inp_sentences):

  if len(inp_sentences.get_shape())==1:
    encoder_input = tf.expand_dims(inp_sentences, 0)
    decoder_input = [french_word2id["<start>"]]
    output = tf.expand_dims(decoder_input, 0)

  else:
    encoder_input = inp_sentences
    decoder_input = [french_word2id["<start>"]]*inp_sentences.get_shape()[0]
    output = tf.expand_dims(decoder_input, -1)


  # encoder_input = tf.expand_dims(inp_sentence, 0)
  
  # decoder_input = [french_word2id["<start>"]]
  # output = tf.expand_dims(decoder_input, 0)
  
  for i in range(pe_target):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,  
                                                 combined_mask,
                                                 dec_padding_mask)
    
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # # return the result if all the seqs has the end token
    if tf.reduce_sum(tf.cast((tf.reduce_sum(tf.cast(output == french_word2id["<eos>"], tf.float32),axis=1)>0), tf.float32)) == inp.get_shape()[0]:
      return output, attention_weights
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  # return tf.squeeze(output, axis=0), attention_weights
  return output, attention_weights

In [16]:
all_preds = []
for (batch_i, (inp, tar)) in tqdm(enumerate(tensor_val)):
  preds, attention = generate_predictions(inp)
  all_preds.append(preds)

In [17]:
translated_sentences = []

for k in tqdm(all_preds):
  for i in k:
    sentence_french = []
    for j in i.numpy()[1:]:
      if j==0 or j==french_word2id["<eos>"]:
        break
      sentence_french.append(french_id2word[j])

    sentence_french = " ".join(sentence_french)

    translated_sentences.append(sentence_french)

translated_sentences = "\n".join(translated_sentences)

with open("predictions.txt","w") as f:
  f.write(translated_sentences)

In [18]:
!pip install sacrebleu
!python evaluator.py --input-file-path ./predictions.txt --target-file-path ./data/split_val.lang2 --do-not-run-model 

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 17.1MB 200kB/s 
final avg bleu score: 11.16


In [19]:
ckpt_save_path = ckpt_manager.save()
print ('Saving checkpoint for epoch {} iteration_i {} at {}'.format(epoch+1,
                                                    iteration_i,
                                                    ckpt_save_path))

Saving checkpoint for epoch 5 iteration_i 22 at ./checkpoints/trainst_3_/ckpt-4


Self-Training Monolingual Data Generation

In [0]:
amount_data_start = 100000
amount_data_end = 200000

In [0]:
with open(data_path/"unaligned_tokenized_rempunc.en","r") as f:
    english_monolingual = f.read().strip().lower()
print(len(english_monolingual.split("\n")), english_monolingual[:200])

474000 for the second phase of the trials we just had different sizes small medium large and extra - large it 's true
geng had been my host the previous january when i was the first us defense secretary to v


In [0]:
def transform_test_data(lang1, dict_word2id, amount_data_start=None,amount_data_end=None):
  lines = lang1.split("\n")
  if amount_data_start or amount_data_end:
    lines = lines[amount_data_start:amount_data_end]
  data = []

  for line in lines:
    line2id = [dict_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(dict_word2id[word])
      except:
        line2id.append(dict_word2id["<unk>"])
    line2id.append(dict_word2id["<eos>"])
    data.append(line2id)

  return data

english_monolingual_data = transform_test_data(english_monolingual, english_word2id, amount_data_start, amount_data_end)
len(english_monolingual_data)

100000

In [0]:
max([len(i) for i in english_monolingual_data])

112

In [0]:
with open(data_path/"unaligned.fr","r") as f:
    french_monolingual = f.read().strip().lower()
print(len(french_monolingual.split("\n")), french_monolingual[:200])
french_monolingual_data = transform_test_data(french_monolingual, french_word2id)
print(len(french_monolingual_data))
max([len(i) for i in french_monolingual_data])

474000 nous n’aurions pas pu dégager d’accord sur un calendrier de conclusion de la cig sans l’engagement politique de mes collègues du conseil européen.
(de) madame la présidente, monsieur le commissaire, m
474000


220

In [0]:
pe_input = max([len(i) for i in english_monolingual_data])
pe_target = max([len(i) for i in french_monolingual_data])

In [0]:
tensor_test = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(english_monolingual_data, padding='post')
)).batch(BATCH_SIZE, drop_remainder=False)

In [0]:
transformer = Transformer(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff, 
    input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size, 
    pe_input=pe_input, pe_target=pe_target, rate=dropout_rate)

In [0]:
experiment_number = "7_smaller_1_1024_"

checkpoint_path = "./checkpoints/train"+experiment_number

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [0]:
all_preds = []
for batch_i, inp in tqdm(enumerate(tensor_test.unbatch().batch(128)),total=len(english_monolingual_data) // 128 + 1):
  preds, attention = generate_predictions(inp)
  all_preds.append(preds)

In [0]:
translated_sentences = []

for k in tqdm(all_preds):
  for i in k:
    sentence_french = []
    for j in i.numpy()[1:]:
      if j==0 or j==french_word2id["<eos>"]:
        break
      sentence_french.append(french_id2word[j])

    sentence_french = " ".join(sentence_french)

    translated_sentences.append(sentence_french)

translated_sentences = "\n".join(translated_sentences)

with open("predictions_english_monolingual_"+str(amount_data_start)+"_"+str(amount_data_end)+".txt","w") as f:
  f.write(translated_sentences)